#  A quick primer on distributed computing Julia

The Julia language natively supports parallel and distributed computing via its `Distributed` package. Julia's distributed computing is based on one-sided communications and remote functions calls. We start by loading the `Distributed` package and by adding two Julia workers to the running session. (Here, all workers are running on our local PC, but if our Julia session was running on a cluster, these workers would be located on the worker nodes.)

In [ ]:
using Distributed
addprocs(2)
@show nprocs();

Packages and libraries in Julia are importet via the `using` statement. To make loaded packages available on all workers and not just the master process, we have to tag our expression with the `@everywhere` macro:

In [ ]:
@everywhere using LinearAlgebra, Random

Defining a function that is callable both locally and on the remote workers works in the same way:

In [ ]:
# Define function everywhere
@everywhere function hello_world(name1, name2; kwargs...)
    print("Hello ", name1, " and ", name2, "\n")
    return "Goodbye from worker $(myid())"
end

Calling the function executes the above code on our (local) master process:

In [ ]:
# Execute function locally
out = hello_world("Bob", "John")
@show out;

To execute the function on a remote worker, we have to use another macro. By calling our function with the `@spawn` statement, we call the function on one of the workers from the current pool. The `@spawn` macro is non-blocking and immediately returns our call with a so-called `Future`, which points to a future result on the remote worker.

In [ ]:
# Execute function on remote worker
future = @spawn hello_world("Bob", "John");
@show typeof(future);

Using the `fetch` function, we can copy the result from the remote process to the local memory. The `fetch` function is blocking and waits until the remote computation has been completed and the result is available.

In [ ]:
out = fetch(future)
@show out;

There are a few other macros and functions available in Julia to execute remote function calls. One of the important ones that we want to look at is the `pmap` function, which takes a custom function and applies it to all elements of an array using the current pool of workers. In the following example, we execute our `hello_world` function twice by looping over the list of input arguments. Unlike `@spawn`, `pmap` is blocking and returns a list of results after all workers have terminted their computations.

In [ ]:
# Parallel map
out = pmap(name -> hello_world(name, "Jane"), ["Bob", "John"])
@show out;

# Remote functions call to Azure Batch with AzureClusterlessHPC.jl

In the next section, we will use a similar set of macros provided by our package `AzureClusterlessHPC.jl` to execute remote function calls via Azure Batch. First, we have to provide user credentials for our Azure Batch and Storage accounts. Furthermore, we read a set of parameters specifying our batch setup; namely the pool name, job name, VM type and number of nodes.

In [ ]:
# Set path to credentials
ENV["CREDENTIALS"] = joinpath(pwd(), "credentials.json")

# Set path to batch parameters (pool id, VM types, etc.)
ENV["PARAMETERS"] = joinpath(pwd(), "parameters.json")

# Load package
using AzureClusterlessHPC;

Next, we start our batch pool and pass it a startup shell script, which optionally let's us include Julia packages that are installed on the nodes in the pool. Here, we don't need any additionally packages and use a default startup script:

In [ ]:
startup_script = "pool_startup_script.sh"
create_pool_and_resource_file(startup_script)

To make code, packages and variables available on remote workers in the batch pool, `AzureClusterlessHPC` provides the `@batchdef` macro. Just as the `@everywhere` macro, it allows us to tag package imports and function definitions:

In [ ]:
@batchdef using LinearAlgebra, Random, Distributed

In [ ]:
@batchdef function hello_batch(myid, name1, name2; kwargs...)
    print("Hello $name1 and $name2 from worker $myid.\n")
    return "Goodbye from worker $myid" 
end;

As before, calling the above function without any macros executes the function on our local PC or VM:

In [ ]:
out = hello_batch(1, "Bob", "John")
@show out;

To execute functions remotely on a batch worker, `AzureClusterlessHPC` provides a similar macro to `@spawn` called `@batchexec`. By executing our function with this macro, our function call is submitted and executed as an Azure Batch job. Like `@spawn`, the macro is non-blocking and returns a batch control panel:

In [ ]:
bctrl = @batchexec hello_batch(1, "Bob", "John")
@show typeof(bctrl);

The batch controller contains some basic information about the batch job such as the pool and job ids and provides some basic functionalities like `terminate_job` or `delete_job`. Furthermore, it has a field `bctrl.output` which contains a Julia Future to the function output. As before, we can copy the result to the local memory using the (blocking) `fetch` function:

In [ ]:
out = fetch(bctrl)
@show out;

We can delete the job by applying the `delete_job` function to the batch controller:

In [ ]:
delete_job(bctrl)

`AzureClusterlessHPC.jl` also provides functionalities to run `pmap` commands with Azure Batch. However, unlike Julia's basic `pmap` function, which automatically executes the function on remote workers, `pmap` has to be tagged with `@batchexec` in order to execute the function call with Azure Batch. (Calling `pmap` without `@batchexec` will execute the function in the local worker pool instead.)

Whereas the basic `pmap` function is a blocking call that returns a list of the worker outputs, calling `pmap` with `@batchexec` is non-blocking and returns a batch controller. Here we execute our `hello_batch` function four times as a multi-task Azure Batch job:

In [ ]:
bctrl = @batchexec pmap(idx -> hello_batch(idx, "Bob", "John"), 1:4);

As before, we can copy the output to the local memory via the `fetch` function. We can either fetch the output of a specific task by calling `fetch(bctrl, 1)` (to fetch the result from task 1), or we call `fetch` without a task id, in which case we wait for all tasks to terminate and fetch their results:

In [ ]:
out = fetch(bctrl); delete_job(bctrl)
@show out;

Return arguments are not limited to simple strings or variables, but they can also included arrays or custom data structures. For example, we can define a custom structure `MyStruct` on the local machine as well as on the batch workers:

In [ ]:
@batchdef struct MyStruct
    a
    b
end;

Now, we define a new function, which returns a double precision array, as well as an instance of our custom class, containing an integer and a single precision array. 

In [ ]:
# Define functions
@batchdef function hello_earth(name1, name2; kwargs...)

    print("Hello ", name1, " and ", name2, "\n")
    print("kwargs: ", kwargs..., "\n")
    
    # Create some random output
    out1 = randn(2,2)
    out2 = MyStruct(4, ones(Float32, 2,3))

    return out1, out2
end;

The `@batchexec` macro supports generic Julia function calls, including optional and keyword arguments.

In [ ]:
# Multi-task batch job via pmap
kwargs = (kw1 = "one", kw2 = "two")
bctrl = @batchexec pmap(name -> hello_earth(name, "Bob"; kwargs...), ["Jane", "John"]);

As in the previous example, we can copy the function output to the local memory using `fetch`. The only restirction is, that the structure/class of the return argument is also known on our local worker (which it is in this case):

In [ ]:
out = fetch(bctrl)
@show out;

To clean up our Azure resources, we apply the `destroy!` function to our batch controller, which deletes the jobs, removes the blob container with temporary files and shuts down the pool:

In [ ]:
destroy!(bctrl);